In [ ]:
print('start contract labels')
import sys
sys.path.append("../helper_functions")
import google_bq_utils as bqu
import clickhouse_utils as ch
sys.path.pop()

import numpy as np
import pandas as pd
import time
import requests as r

In [ ]:
gtp_contracts_api = 'https://api.growthepie.xyz/v1/contracts.json'
table_name = 'contract_labels_growthepie'

In [ ]:
# Fetch data from API
response = r.get(gtp_contracts_api)
data = response.json()

df = pd.DataFrame(data)

# Force addresses to lowercase
df['address_original'] = df['address']
df['address'] = df['address'].str.lower()

In [ ]:
ch.write_df_to_clickhouse(df, table_name, if_exists='replace')

In [ ]:
bqu.write_df_to_bq_table(df, table_name)